In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import ImageNormalize, MinMaxInterval, ZScaleInterval, LogStretch
import numpy.ma as ma
from astropy.io import fits

import src.astrocal.master as ac
import src.astrocal.reduction as re
import src.astrocal.astrometry as am
import src.astrocal.photometry as pt

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

In [ ]:
main = "C:/Users/aurel/Desktop/DUAO Projet/(6) Hebe"

## Master frames

In [ ]:
savedir_dark = f"{main}/DARK/MASTER"
savedir_flat = f"{main}/FLAT/MASTER"
savedir_mask = f"{main}/MASK"

In [ ]:
dark_filename = f"{main}/DARK/DARK*"
master_dark, mask_hot = ac.make_master_dark(dark_filename, gen_hot_px=True, threshold=3, clean_bad_px=False, verbose=True)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,12))
norm = ImageNormalize(master_dark.data, interval=ZScaleInterval())
axs[0].imshow(master_dark.data, cmap='gray', origin='lower', norm=norm)
axs[1].imshow(mask_hot, cmap='gray_r', origin='lower', vmin=0, vmax=1)
plt.show()

In [ ]:
dark_sample = glob.glob(dark_filename)[0].replace('\\','/')
ac.save_master(savedir_dark, master_dark, dark_sample, out_name=None)
ac.save_mask(savedir_mask, mask_hot, dark_sample, out_name=None)

In [ ]:
flat_filename = f"{main}/FLAT/SDSSgp+/FLAT*"
master_flat, mask_dead = ac.make_master_flat(flat_filename, threshold=0.5, gen_dead_px=True, clean_bad_px=False, mask=mask_hot, verbose=True)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(12,12))
norm = ImageNormalize(master_flat.data, interval=ZScaleInterval())
axs[0].imshow(master_flat.data, cmap='gray', origin='lower', norm=norm)
axs[1].imshow(mask_dead, cmap='gray_r', origin='lower', vmin=0, vmax=1)
plt.show()

In [ ]:
flat_sample = glob.glob(flat_filename)[0].replace('\\','/')
ac.save_master(savedir_flat, master_flat, flat_sample, out_name=None)
ac.save_mask(savedir_mask, mask_dead, flat_sample, out_name=None)

## Image reduction

In [ ]:
date = "20240605"
filter = "SDSSip+"

light_filename = f"{main}/LIGHT/{date}/{filter}/*.fits"
savedir = "/".join(light_filename.split("/")[:-1]) + "/REDUCED"
master_dark = glob.glob(f"{main}/DARK/MASTER/*")[0]
master_flat = glob.glob(f"{main}/FLAT/MASTER/*{filter}*")[0]
mask_dark = glob.glob(f"{main}/MASK/*DARK*")[0]
mask_flat = glob.glob(f"{main}/MASK/*FLAT*{filter}*")[0]

In [ ]:
raw = fits.open(glob.glob(light_filename)[0])[0]
reduced = re.reduce_one(glob.glob(light_filename)[0], fits.open(master_dark)[0].data, fits.open(master_flat)[0].data, 
                        mask=np.logical_or(fits.open(mask_dark)[0].data.astype(bool), fits.open(mask_flat)[0].data.astype(bool)),
                        verbose=True)

fig, axs = plt.subplots(1,2,figsize=(12,12))
norm = ImageNormalize(raw.data, interval=ZScaleInterval())
axs[0].imshow(raw.data, cmap='gray', origin='lower', norm=norm)
norm = ImageNormalize(reduced.data, interval=ZScaleInterval())
axs[1].imshow(reduced.data, cmap='gray', origin='lower', norm=norm)
plt.show()

In [ ]:
re.reduce(light_filename, savedir, master_dark, master_flat, mask_name=[mask_dark, mask_flat], verbose=True)

## Astrometric calibration (plate-solving)

In [ ]:
date = "20240531"
filter = "SDSSgp+"

generic_filename = f"{main}/LIGHT/{date}/{filter}/REDUCED/*.fits"
savedir = f"{main}/LIGHT/{date}/{filter}/REDUCED"
mask_dark = glob.glob(f"{main}/MASK/*DARK*")[0]
mask_flat = glob.glob(f"{main}/MASK/*FLAT*{filter}*")[0]
am.astrometry_all(generic_filename, savedir, overwrite=True, mask_name=[mask_dark, mask_flat], first=0, verbose=True)

In [ ]:
date = "20240531"
filter = "SDSSrp+"

generic_filename = f"{main}/LIGHT/{date}/{filter}/REDUCED/*.fits"
savedir = f"{main}/LIGHT/{date}/{filter}/REDUCED"
mask_dark = glob.glob(f"{main}/MASK/*DARK*")[0]
mask_flat = glob.glob(f"{main}/MASK/*FLAT*{filter}*")[0]
am.astrometry_all(generic_filename, savedir, overwrite=True, mask_name=[mask_dark, mask_flat], first=0, fwhm=8, threshold=5, verbose=True)

In [ ]:
date = "20240531"
filter = "SDSSip+"

generic_filename = f"{main}/LIGHT/{date}/{filter}/REDUCED/*.fits"
savedir = f"{main}/LIGHT/{date}/{filter}/REDUCED"
mask_dark = glob.glob(f"{main}/MASK/*DARK*")[0]
mask_flat = glob.glob(f"{main}/MASK/*FLAT*{filter}*")[0]
am.astrometry_all(generic_filename, savedir, overwrite=True, mask_name=[mask_dark, mask_flat], fwhm=8, threshold=5, first=0, verbose=True)

## Photometric calibration

In [ ]:
date = "20240531"
filter = "SDSSgp+"

mask_dark = glob.glob(f"{main}/MASK/*DARK*")[0]
mask_flat = glob.glob(f"{main}/MASK/*FLAT*{filter}*")[0]
mask_name=[mask_dark, mask_flat]

generic_filename = f"{main}/LIGHT/{date}/{filter}/REDUCED/*.fits"

savedir = f"{main}/LIGHT/{date}/{filter}/PHOTOMETRY"

pt.photometry_all(generic_filename, savedir, mask_name,
                  calib_mag_col=f"{filter[-3]}mag", calib_emag_col=f"e_{filter[-3]}mag", 
                  plot=False, verbose=True)

In [ ]:
date = "20240531"
filter = "SDSSrp+"

mask_dark = glob.glob(f"{main}/MASK/*DARK*")[0]
mask_flat = glob.glob(f"{main}/MASK/*FLAT*{filter}*")[0]
mask_name=[mask_dark, mask_flat]

generic_filename = f"{main}/LIGHT/{date}/{filter}/REDUCED/*.fits"

savedir = f"{main}/LIGHT/{date}/{filter}/PHOTOMETRY"

pt.photometry_all(generic_filename, savedir, mask_name, first=0,
                  detect_fwhm=8, detect_threshold=5,
                  calib_mag_col=f"{filter[-3]}mag", calib_emag_col=f"e_{filter[-3]}mag", 
                  plot=False, verbose=True)

In [ ]:
date = "20240531"
filter = "SDSSip+"

mask_dark = glob.glob(f"{main}/MASK/*DARK*")[0]
mask_flat = glob.glob(f"{main}/MASK/*FLAT*{filter}*")[0]
mask_name=[mask_dark, mask_flat]

generic_filename = f"{main}/LIGHT/{date}/{filter}/REDUCED/*.fits"

savedir = f"{main}/LIGHT/{date}/{filter}/PHOTOMETRY"

pt.photometry_all(generic_filename, savedir, mask_name,
                  detect_fwhm=8, detect_threshold=5,
                  calib_mag_col=f"{filter[-3]}mag", calib_emag_col=f"e_{filter[-3]}mag", 
                  plot=False, verbose=True)